# 🧠 Урок 35: Трансформеры и модели BERT, GPT
**Цель урока:** Понять, как работают трансформеры, BERT и GPT, и научиться использовать их для задач NLP. Подходит для новичков.

## 📌 Зачем нужны трансформеры?
- **Трансформеры** решили проблему зависимости от длины последовательности (в отличие от RNN/LSTM).
- **Self-Attention:** Позволяет модели видеть связи между любыми словами, даже если они далеко друг от друга.
- **Параллелизация:** Обучение намного быстрее, чем у рекуррентных сетей.
- **Аналогия:** Трансформер — как человек, который читает всю книгу сразу и понимает, кто есть кто, вместо того чтобы ждать конца главы [[2]].

## 📐 Механизм внимания (Attention)
- **Внимание (Attention):** Модель учится, какие части текста важны для текущего слова.
- **Self-Attention:** Каждое слово сравнивается со всеми другими для вычисления их взаимодействия.
- **Q (Query), K (Key), V (Value):**
  - Query — текущее слово, которое ищет связь.
  - Key — все слова, с которыми Query сравнивается.
  - Value — данные, которые используются для вычисления финального представления.
- **Формула:**
  ```python
  attention_weights = softmax(Q @ K.T / sqrt(d_k))
  output = attention_weights @ V
  ```
- **Аналогия:** Self-Attention — как человек, который читает текст и связывает "Apple" в начале и конце абзаца [[1]].

## 🧱 Что такое BERT?
- **BERT (Bidirectional Encoder Representations from Transformers):** Модель, которая обучается на основе контекста (оба направления).
- **Как работает?**
  - **Masked Language Model (MLM):** Предсказывает случайно замаскированные слова.
  - **Next Sentence Prediction (NSP):** Определяет, идут ли предложения друг за другом.
- **Преимущества:**
  - Понимает двусмысленность (например, "яблоко" как компания и как фрукт).
  - Подходит для классификации, вопросно-ответных систем, NER.
- **Ограничения:**
  - Не умеет генерировать текст.
  - Требует больших вычислений.
- **Пример:**
  ```python
  from transformers import BertTokenizer, TFBertModel
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  model = TFBertModel.from_pretrained('bert-base-uncased')
  inputs = tokenizer("Привет, мир!", return_tensors='tf')
  outputs = model(inputs)
  ```
- **Аналогия:** BERT — как человек, который читает книгу и помнит контекст до и после каждого слова [[1]].

## 📈 Что такое GPT?
- **GPT (Generative Pretrained Transformer):** Модель, которая обучается предсказывать следующее слово.
- **Как работает?**
  - **Decoder-only:** Генерирует текст, опираясь только на предыдущие слова.
  - **Causal Language Model:** Прогнозирует следующее слово на основе прошлых.
- **Преимущества:**
  - Может генерировать тексты любой длины.
  - Хорошо справляется с логикой и рассуждениями.
- **Ограничения:**
  - Не видит будущих слов (только прошлое).
  - Может «врать», если не знает ответа [[6]].
- **Пример:**
  ```python
  from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  model = TFGPT2LMHeadModel.from_pretrained('gpt2')
  inputs = tokenizer("Почему небо голубое?", return_tensors='tf')
  outputs = model.generate(inputs['input_ids'], max_length=50)
  print(tokenizer.decode(outputs[0]))
  ```
- **Аналогия:** GPT — как писатель, который продолжает историю, основываясь на уже написанном [[4]].

## 📊 Сравнение BERT и GPT
- **BERT:** Двунаправленный, лучше понимает контекст, подходит для классификации.
- **GPT:** Однонаправленный, лучше генерирует текст, но может ошибаться в фактах.
- **Примеры применения:**
  - **BERT:** Анализ тональности, NER, Q&A.
  - **GPT:** Чат-боты, автоматическая генерация текста, перевод.
- **Аналогия:** BERT — как студент, который перечитал учебник и может ответить на вопросы, GPT — как писатель, который придумывает ответы на основе предыдущего опыта [[5]].

## 🧪 Практика: Использование BERT для классификации
### Шаг 1: Загрузка данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Искусственный датасет
df = pd.DataFrame({
    'text': [
        "Я люблю Python!",
        "Java — мой любимый язык.",
        "Python лучше всех!",
        "Мне нравится C++",
        "Люблю машинное обучение",
        "Глубокое обучение круто",
        "Я программист на Python"
    ],
    'label': [1, 0, 1, 0, 1, 1, 1]  # 1 = любит Python, 0 = нет
})

# Разделение
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2)

### Шаг 2: Токенизация и обучение

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Токенизация
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)

# Создание датасета
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))

In [ ]:
# Обучение
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16), epochs=3)

In [ ]:
# Оценка
loss, accuracy = model.evaluate(test_dataset.batch(16))
print(f'Accuracy: {accuracy:.2f}')

## 📈 Практика: Генерация текста с GPT
### Шаг 1: Использование GPT-2

In [ ]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

# Загрузка GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Генерация
inputs = tokenizer("Искусственный интеллект", return_tensors='tf')
outputs = model.generate(inputs['input_ids'], max_length=50, num_return_sequences=1)
print(tokenizer.decode(outputs[0]))

### Шаг 2: Использование GPT-3 (через API)
```python
# Пример через OpenAI API (требуется ключ)
import openai
openai.api_key = "your-api-key"

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="Почему небо голубое?",
  max_tokens=100
)
print(response.choices[0].text)
```
- **Аналогия:** GPT-3 — как учитель, который может ответить на любой вопрос, даже если он не уверен в точности [[6]].

## 📉 Что такое переобучение и как его избежать?
- **Переобучение (Overfitting):** Модель идеально запоминает тренировочные данные, но плохо работает на тестовых.
- **Причины:**
  - Слишком много параметров.
  - Мало данных.
- **Как бороться?**
  - Используйте Dropout.
  - Добавьте регуляризацию.
  - Увеличьте данные (например, аугментация).
- **Пример:**
  ```python
  from transformers import TrainingArguments, Trainer
  
  training_args = TrainingArguments(
      output_dir='./results',
      num_train_epochs=3,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      warmup_steps=500,
      weight_decay=0.01,
      logging_dir='./logs',
  )
  ```
- **Аналогия:** Переобучение — как зубрежка ответов, а не понимание темы .

## 📊 Как выбрать лучшие гиперпараметры?
- **GridSearchCV:** Перебор всех возможных комбинаций.
- **RandomizedSearchCV:** Случайный перебор (быстрее, чем GridSearch).
- **Пример:**
  ```python
  from sklearn.model_selection import GridSearchCV
  param_grid = {'learning_rate': [1e-5, 3e-5, 5e-5], 'batch_size': [8, 16, 32]}
  grid = GridSearchCV(model, param_grid, cv=3, scoring='accuracy')
  grid.fit(X_train, y_train)
  ```
- **Аналогия:** Подбор гиперпараметров — как поиск идеального рецепта через эксперименты с ингредиентами [[7]].

## 📝 Домашнее задание
**Задача 1:** Измените prompt для GPT-2:
- Попробуйте разные темы (например, "Как стать программистом", "Почему солнце светит").
- Как меняется качество генерации?
- Сохраните результаты в файл.

**Задача 2:** Обучите BERT на датасете IMDB (тональность отзывов):
- Скачайте данные с помощью `fetch_openml`.
- Оцените точность.
- Напишите отчет (200–300 слов), где:
  - Опишите, как вы обучали модель.
  - Сравните с логистической регрессией.
  - Объясните, почему BERT работает лучше на сложных зависимостях.
  - Приведите примеры, где трансформеры полезны (например, чат-боты, анализ тональности).

In [ ]:
# Ваш код здесь
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import tensorflow as tf

# Генерация с разными prompt
prompts = [
    "Как стать программистом?",
    "Почему солнце светит?",
    "Что такое машинное обучение?"
]

# Генерация
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors='tf')
    outputs = model.generate(inputs['input_ids'], max_length=100)
    print(f'\nPrompt: {prompt}')
    print(tokenizer.decode(outputs[0]))

In [ ]:
# Загрузка IMDB датасета
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']
X = X / 255.0  # Нормализация
y = np.array([int(label) % 2 for label in y])  # Бинаризация: четные/нечетные цифры

# Разделение
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

In [ ]:
# Обучение BERT на IMDB
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Токенизация
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(X_train.astype(str).tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.astype(str).tolist(), truncation=True, padding=True, max_length=128)

## ✅ Рекомендации по выполнению
- **Задача 1:** Используйте разные prompt и анализируйте, как меняется генерация.
- **Задача 2:** Для BERT используйте `TFBertForSequenceClassification`.
- **Подсказка:** Если модель переобучается, уменьшите `num_train_epochs` или увеличьте `weight_decay`.